# Intern overlapp i trafikkmengde-data

For interne analyseformål skal vi finne NVDB-objekter som er stedfestetpå samme sted 
på en eller flere lenkesekvenser. Objektene skal ha samme verdi for egenskapen _"År, gjelder for"_ 
og selvsagt også overlappende gyldighetsdatoer. Dvs to objekter stedfestet på samme sted på vegnettet er OK hvis forekomstene gjelder for ulike år. 

Jeg velger å gjøre analysen med Pandas-biblioteket i python. Prinsippet er enkelt: Finn overlapp mellom tabell A og en kopi av A, med regler som ekskluderer overlapp med samme rad, pluss spesialregelen om at egenskapstypen _År, gjelder for_ skal ha samme verdi. Overlapp er definert som overlapp i tid pluss overlapp i posisjon langs lenkesekvens. 

Merk at denne metoden gir symmetriske overlapp: Hvis rad 13 overlapper med rad 146, så vil du også finne overlapp mellom rad 146 og 13. Dette har jeg ikke luket ut fra datasettet. Tvert i mot er et en fordel når du utforsker data i kart at du ser både rad 13 og 146 i kartet... 

### Datakilde = TNE 

Ettersom vi p.t. mangler mulighet for lettvint dump av all tilgjengelig historikk for en 
vegobjekttype (evt avgrenset på fra- og tildato) så bruker vi den interne datakilden TNE. 
(Joda, vi kjenner til en workaround hvor vi [iterer over datoer](github.com/LtGlahn/workinprogress/blob/historisk-riksveg/README_hentrv532.md#nedlasting-av-data) for å hente alle  historiske data, men det er litt masete og tidkrevdende).  
I stedet er det for meg kjappere å ta en tilsvarende dump fra det interne systemet TNE (tar ca 40 minutt). 
Dette er en GIS-plattform (arc SDE), og det innebærer at data må ombarbeides en del for å kunne 
utnytte denne plattformens styrker (og unngå en del svakheter). 

Her er de viktiste TNE-særhetene

  * Data blir segmentert, dvs brutt opp i småbiter med en forekomst i TNE per lenkesekvens-bit. 
  * Stedfesting på Vegtrasé - topologinivå blir også kopiert til topologinivåene kjørebane- og kjørefelt. 
  * Topologinivåene i TNE angis med 0 = VT, 1 = VTKB, 2 = Kjørebane, 3 = Kjørefelt. 
  * Egenskapsnavnene er en ascii-forkortelse etterfulgt av ID til egenskapstypen, eks AR_4621 for egenskapstypen _4621 År, gjelder for_.
  * Kolonnen _NETOBJECTTV_ID_ er satt sammen av  objektID og versjonsID, med semikolon imellom. 
  * ROUTE_ID = Veglenkesekvens ID, MEASURE = posisjon langs veglenkesekvens. 
  * Relasjoner blir ikke håndtert spesielt bra i TNE 2.7. Kolonnene _BA_<løpenr>_ skal liksom inneholde ID til datterobjekt, men her er det noe kluss med håndtering av lister. 
  * Datoer er håndtert som heltall mellom 19500101 og 99991231 
 
 
# Resultater 

Fant 2 * 6266 overlapp. Ettersom jeg ikke har filtrert vekk symmetri så er det  12532 rader i datasettet, som kan lastes ned [her](https://github.com/LtGlahn/workinprogress/blob/master/aadtoverlapp.gpkg?raw=true). Mitt favorittformat er [OGC Geopackage](https://www.geopackage.org/), en sqlite-database hvor vi følger standarder for geografisk informasjon, men jeg kan lettvint dumpe til andre formater (csv, rein sqlite osv). 
 
 
 ![Alle 1253 radene](./pics/aadtoverlapp_norge.png)
 
 
 Her er et eksempel med 4 meter overlapp: Blå tverrlinje er endepunktet på det øverste (nordligste) objektet `84811029:1`, mens den grønne linja er startpunktet på objekt `84811028:1`. 
 
![Alle 1253 radene](./pics/overlapp1.png)

Dette eksemplet er for strekningen Børsa-Buvika, totalt ca 9 km fordelt på 2 objekter med 4.4m overlapp, gjelder for året 1989. 
![Alle 1253 radene](./pics/overlapp1kart.png)


# Analyseprosess: Laster bibliotek 

In [1]:
import re
import pdb
from copy import deepcopy
import sqlite3

from shapely import wkt 
from shapely import geometry 
# from shapely.ops import unary_union
import pandas as pd 
import geopandas as gpd 
from datetime import datetime

import STARTHER
import nvdbapiv3
from nvdbapiv3 import apiforbindelse

Legger NVDB api til søkestien


# Leser data

In [2]:
t0 = datetime.now()
# A = gpd.read_file( 'trafikkmengde_idag.gpkg' )
A = gpd.read_file( 'trafikkmengde_alle.gpkg' )
t1 = datetime.now()
dt = t1 - t0
print( 'Datainnlesning:', dt.total_seconds()/60, 'minutt' )

/home/jan/anaconda3/envs/geo2021aprV2/lib/python3.9/site-packages/geopandas/geodataframe.py:577: RuntimeWarning: Sequential read of iterator was interrupted. Resetting iterator. This can negatively impact the performance.
  for feature in features_lst:


Datainnlesning: 6.2916494 minutt


# Data prepp 

In [3]:
A['FROM_MEASURE'] = A['FROM_MEASURE'].round( 8)
A['TO_MEASURE'] = A['TO_MEASURE'].round( 8)

In [4]:
A['nvdbId'] = A['NETOBJECTTV_ID'].apply( lambda x : x.split(':')[0] )
A['versjon'] = A['NETOBJECTTV_ID'].apply( lambda x : x.split(':')[1] )

In [5]:
col = [ 'nvdbId', 'versjon', 'NETOBJECTTV_ID', 'FROM_DATE', 'TO_DATE', 
        'SEQ_NO', 'TADT_4623', 'LANGX_4624', 'AR_4621', 'OPFR_6843', 'GRLG_4625', 
        'ANID_4620', 'TRAFA_7477', 'TPPM_5219', 'TPSD_5222', 'FADT_5220', 'FLNG_5221', 
        'KPAR_5223', 'MFAK_4622', 'AADTS_7475', 'AADTS_7476', 'VDB_I_7804', 'ROUTE_ID', 
        'FROM_MEASURE', 'TO_MEASURE', 'TOPOLOGY_LEVEL', 'geometry' ]

In [6]:
# Redusert datasett for utvikling / test / debug
# A = A.iloc[0:20].copy()

In [7]:
# Gjør om til pandas dataframe og konverterer geometriobjekt => Well Known Text
# (fordi join'en vår klager og sutrer når den møter et geometriobjekt i datasettet)
A = pd.DataFrame( A )
A['geometry'] = A['geometry'].apply( lambda x: x.to_wkt() if isinstance( x, geometry.linestring.LineString) else '')

In [8]:
# Dobbeltsjekker at OBJECTID er unik. Dette er primærnøkkel i TNE-tabellen, så den bør helst være unik... 
dup =  len( A[ A['OBJECTID'].duplicated() ] )
if dup == 0: 
    print( 'Vi har ', dup, 'duplikater og kan bruke OBJECTID som unik ID' )
else:
    print( 'PASS PÅ - vi har ', dup, 'duplikater og kan IKKE bruke OBJECTID som unik ID' )
          

Vi har  0 duplikater og kan bruke OBJECTID som unik ID


I TNE brukes _OBJECTID_ som primærnøkkel, så hvis den ikke er unik er noe alvorlig gærnt. 

### Dupliserer datasett

Vi skal finne datasettets overlapp med seg selv. Dette løses best ved å finne overlapp mellom A og B hvor B er en kopi av A, men med regler som eksluderer et objekts overlapp med seg selv. Pluss selvsagt de andre reglene vi har (samme verdi for _'År, gjelder for'_, og overlapp i tid og rom)

In [9]:
B = A.copy()
B = B.add_prefix( 'b_')

In [10]:
t2 = datetime.now()
dt2 = t2 - t1
print( 'Kjøretid data prepp:', dt2.total_seconds() / 60, 'minutt')

Kjøretid data prepp: 15.626001166666667 minutt


# Finner overlapp

Kolonnedefinisjon for SQL spørring:

In [11]:
col_vlinkA  = 'ROUTE_ID' 
col_startA  = 'FROM_MEASURE'
col_sluttA  = 'TO_MEASURE'
col_t0A     = 'FROM_DATE'
col_t1A     = 'TO_DATE'

col_vlinkB  = 'b_ROUTE_ID' 
col_startB  = 'b_FROM_MEASURE' 
col_sluttB  =  'b_TO_MEASURE'
col_t0B     = 'b_FROM_DATE'
col_t1B     = 'b_TO_DATE'



In [12]:
qry = ( f"select * from A\n"
                f"INNER JOIN B ON\n"
                f"A.OBJECTID != B.b_OBJECTID and\n"
                f"A.AR_4621 = B.b_AR_4621 and\n"
                f"A.{col_vlinkA} = B.{col_vlinkB} and\n"
                f"A.{col_startA} < B.{col_sluttB} and\n"
                f"A.{col_sluttA} > B.{col_startB} and\n"
                f"A.{col_t0A} < B.{col_t1B} and\n" 
                f"A.{col_t1A} > B.{col_t0B}"
            )

print( qry )

select * from A
INNER JOIN B ON
A.OBJECTID != B.b_OBJECTID and
A.AR_4621 = B.b_AR_4621 and
A.ROUTE_ID = B.b_ROUTE_ID and
A.FROM_MEASURE < B.b_TO_MEASURE and
A.TO_MEASURE > B.b_FROM_MEASURE and
A.FROM_DATE < B.b_TO_DATE and
A.TO_DATE > B.b_FROM_DATE


In [13]:
conn = sqlite3.connect( ':memory:')
A.to_sql( 'A', conn, index=False )
B.to_sql( 'B', conn, index=False )
joined = pd.read_sql_query( qry, conn )

In [14]:
t3 = datetime.now()
dt3 = t3 - t2
print( 'Kjøretid join:', dt3.total_seconds()/60, 'minutter')

Kjøretid join: 33.87213143333334 minutter


# Hvor stor overlapp har vi? 

Basert på vegnettposisjoner (_FROM_MEASURE, TO_MEASURE_ ) kan vi lettvint finne om vi har overlapp i hele dette segmentets utstrekning, evt hvor stor andel overlapp vi har. Og ut fra geometriens lengde vet vi da også overlapp-lengden. 


Først litt Lager en GeoDataFrame basert på Well Known Text - verdien, slik at vi har geografiske objekt, ikke tekst. Dernest finner vi lengden av geometrien, andel overlapp og geometrisk lengde for overlapp. 



In [15]:
joined['geometry'] = joined['geometry'].apply( lambda x: wkt.loads(x) if x != '' else geometry.LineString())

minGdf = gpd.GeoDataFrame( joined, geometry='geometry', crs=5973 )    

In [33]:
minGdf['geometrilengde'] = minGdf['geometry'].length

In [34]:
minGdf.geometrilengde.describe()

count    12532.000000
mean       307.135711
std       1091.405277
min          0.000000
25%         17.195740
50%         47.833469
75%        131.092365
max      13785.475991
Name: geometrilengde, dtype: float64

In [77]:
def finnOverlappLengde( rad, meterlengde = False ): 
    """
    Finner andel overlapp basert på veglenkeposisjoner. Skal være et tall mellom 
    0 og 1, hvor 1 = overlapp i hele utstrekningen. 
    """
    overlapp_measure =  min( rad['TO_MEASURE'], rad['b_TO_MEASURE'] ) - \
                        max( rad['FROM_MEASURE'], rad['b_FROM_MEASURE'] ) 
                        
    return overlapp_measure / ( rad['TO_MEASURE'] - rad['FROM_MEASURE']  )


In [58]:
minGdf['overlappandel'] = minGdf.apply( finnOverlappLengde, axis=1)

In [61]:
minGdf['overlapplengde'] = minGdf['geometrilengde'] * minGdf['overlappandel']

In [80]:
minGdf.to_file( 'aadtoverlapp.gpkg', layer='aadtoverlapp', driver='GPKG')

# Kolonner i datasettet: 

In [66]:
for ii in minGdf.columns:
    print( ii)

OBJECTID
NETOBJECTTV_ID
FROM_DATE
TO_DATE
SEQ_NO
BA_200670_OID
BA_201812_OID
TADT_4623
LANGX_4624
AR_4621
OPFR_6843
GRLG_4625
ANID_4620
TRAFA_7477
TPPM_5219
TPSD_5222
FADT_5220
FLNG_5221
KPAR_5223
MFAK_4622
AADTS_7475
AADTS_7476
VDB_I_7804
ROUTE_ID
FROM_MEASURE
TO_MEASURE
TOPOLOGY_LEVEL
SHAPE_LEN
geometry
nvdbId
versjon
b_OBJECTID
b_NETOBJECTTV_ID
b_FROM_DATE
b_TO_DATE
b_SEQ_NO
b_BA_200670_OID
b_BA_201812_OID
b_TADT_4623
b_LANGX_4624
b_AR_4621
b_OPFR_6843
b_GRLG_4625
b_ANID_4620
b_TRAFA_7477
b_TPPM_5219
b_TPSD_5222
b_FADT_5220
b_FLNG_5221
b_KPAR_5223
b_MFAK_4622
b_AADTS_7475
b_AADTS_7476
b_VDB_I_7804
b_ROUTE_ID
b_FROM_MEASURE
b_TO_MEASURE
b_TOPOLOGY_LEVEL
b_SHAPE_LEN
b_geometry
b_nvdbId
b_versjon
geometrilengde
overlappandel
overlapplengde


# Litt mer utforsking av datasettet

In [84]:
print( f"Antall rader{len( minGdf)} antall overlapp-treff { len( minGdf) / 2} (symmetrisk)") 

Antall rader12532 antall overlapp-treff 6266.0 (symmetrisk)


In [88]:
minGdf['FROM_DATE'].describe().round()

count       12532.0
mean     20014001.0
std        111856.0
min      19710101.0
25%      19900101.0
50%      19990101.0
75%      20100101.0
max      20210201.0
Name: FROM_DATE, dtype: float64

count    1.253200e+04
mean     2.001400e+07
std      1.118565e+05
min      1.971010e+07
25%      1.990010e+07
50%      1.999010e+07
75%      2.010010e+07
max      2.021020e+07
Name: FROM_DATE, dtype: float64